<a href="https://colab.research.google.com/github/sarthakkapila/AI-ML-ES1701/blob/main/FineTuningBark_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install the required libraries

In [ ]:
!pip install transformers
!pip install torch
!pip install ipython
!pip install datasets
!pip install huggingface_hub
!pip install autotrain-advanced

### Importing necessary libraries

In [52]:
import torch
from transformers import AutoProcessor, AutoModelForTextToWaveform, BarkModel, AutoModelWithLMHead, AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, Trainer, AutoModelForSequenceClassification, TrainingArguments
import IPython.display as ipd

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset

In [ ]:
# Fuck GoogleColab 🖕🏻
device = "cuda" if torch.cuda.is_available() else "cpu"

### Loading model

In [47]:
processor = AutoProcessor.from_pretrained("suno/bark-small")
model = BarkModel.from_pretrained("suno/bark-small")
model_name = "suno/bark-small"

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


### Using model

In [26]:
voice_preset = "v2/en_speaker_6"
inputs = processor("""My fellow Americans, I want to speak to you tonight about
 the troubling events  of the past week. As I have said, the incursion of the US Capitol struck at the very heart of our Republic"""
, voice_preset=voice_preset)

audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()

ipd.Audio(audio_array, rate=22050)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [34]:
dataset = load_dataset("michaelnetbiz/Kendex")
train_dataset = dataset["train"]
train_dataset[0]
train_dataset[1]

train_dataset.features

{'audio': {'path': 's2214pm_2_corinthians_1_2_2022_0403_1626.93.wav',
  'array': array([ 3.35693359e-04,  1.03759766e-03,  6.10351562e-05, ...,
         -6.59179688e-03, -5.40161133e-03, -5.49316406e-03]),
  'sampling_rate': 16000},
 'file': 's2214pm_2_corinthians_1_2_2022_0403_1626.93.wav',
 'text': ' He said in verse 14, also you have acknowledged us in part',
 'duration': 6.05,
 'normalized_text': ' he said in verse fourteen, also you have acknowledged us in part'}

### Tokenizing

In [36]:
tokenized_1 = processor(dataset["train"]["normalized_text"])
tokenized_1

{'input_ids': tensor([[26350,   117,   146,  ...,     0,     0,     0],
        [10357, 12415, 10106,  ...,     0,     0,     0],
        [  146, 23763, 10114,  ...,     0,     0,     0],
        ...,
        [10479, 10393, 20378,  ...,     0,     0,     0],
        [10357, 10124,   119,  ...,     0,     0,     0],
        [14015, 10104, 60582,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

### This is not very efficent when it comes to dictionary so, we will use the Dataset.map() method.

In [39]:
tokenized = processor(
    dataset["train"]["normalized_text"],
    truncation=True,
)

In [43]:
def tokenizer(example):
    return processor(example["normalized_text"], truncation=True)

tokenized_datasets = dataset.map(tokenizer, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=processor)

In [58]:
training_args = TrainingArguments("test-trainer")
model_train = AutoModelForTextToWaveform.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
trainer = Trainer(
    model_train,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=processor,
)

In [ ]:
trainer.train()